## Importuj wymagane biblioteki
Zaimportuj biblioteki PyTorch i transformers potrzebne do ładowania i używania modelu Phi-4.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Ustawienie Losowego Ziarna
Ustaw losowe ziarno, aby zapewnić powtarzalne wyniki w różnych uruchomieniach.


In [ ]:
torch.random.manual_seed(0)

## Załaduj model Phi-4-mini-flash-reasoning i tokenizer
Załaduj model Microsoft Phi-4-mini-flash-reasoning oraz odpowiadający mu tokenizer z Hugging Face. Model zostanie załadowany na CUDA, aby przyspieszyć proces wnioskowania.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Przygotuj wiadomość wejściową
Utwórz wiadomość rozmowy zawierającą zadanie matematyczne dotyczące równania kwadratowego i sformatuj ją, korzystając z szablonu czatu dla modelu.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## Generowanie odpowiedzi
Generuj odpowiedź modelu, korzystając z określonych parametrów, takich jak temperatura i top_p, aby kontrolować losowość w wynikach.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## Dekodowanie Wyniku na Tekst
Przekształć wygenerowane sekwencje tokenów z powrotem na czytelny dla człowieka tekst, pomijając oryginalne tokeny wejściowe, aby pokazać wyłącznie odpowiedź modelu.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za autorytatywne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
